# Worksheet 05

Name:Calvin Li  
UID: U51621195

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

1) centroid 1: [0, 0.5]
   centroid 2: [1.5, 2, 6, 6.5, 7]
2) centroid 1: (0 + 0.5) / 2 = 0.25
   centroid 2: (1.5 + 2 + 6 + 6.5 + 7) / 5 = 4.6
3) centroid 1: [0, 0.5, 1.5, 2]
   centroid 2: [6, 6.5, 7] 
4) centroid 1: (0 + 0.5 + 1.5 + 2) / 4 = 4/4 = 1
   centroid 2: (6 + 6.5 + 7) / 3 = 6.5
   
[1, 6.5]


b) Describe in plain english what the cost function for k means is.

way to evaluate and compare solutions

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

It could be due to the initial centroids since it could converge to the local minimum of the cost function instead of the global minimum and thus causing it to be a different cluster. 

d) Does Lloyd's Algorithm always converge? Why / why not?

Lloyd's algorithm always converges but it could converge to a local minimum instead of the global minimum because Lloyd's algorithm either keeps the cost the same or lowers the cost. 

e) Follow along in class the implementation of Kmeans

In [5]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))


    def is_unassigned(self, i):
        return self.assignment[i] == -1
    
    def unassign_all(self):
        self.assignment = [-1 for _ in range(len(self.data))]
        
    def initialize(self):
        return self.data[np.random.choice(range(len(self.data)), size=self.k, replace=False)]
    
    def are_centers_diff(self, c1, c2):
        for i in range(len(c1)):
            if c1[i] not in c2:
                return True
        return False
    
    def assign(self, centers):
        for i in range(len(self.data)):
            self.assignment[i] = 0
            temp_assign = 0
            temp_dist = self.dist(self.data[i], centers[0])
            for j in range(1, len(centers)):
                new_dist = self.dist(self.data[i], centers[j])
                if temp_dist > new_dist:
                    self.assignment[i] = j
                    temp_dist = new_dist
    
    def calculate_new_centers(self):
        centers = []
        for j in range(self.k):
            cluster_j = self.data[
                np.array([i for i in range(len(self.data)) if self.assignment[i] == j])
            ]
            centers.append(np.mean(cluster_j,axis=0))
        
        return np.array(centers)

    def dist(self, x, y):
        return sum((x - y) ** 2) ** (1/2)

    def lloyds(self):
        centers = self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers = self.calculate_new_centers()
        while self.are_centers_diff(centers, new_centers):
            centers = new_centers
            self.snap(centers)
            self.unassign_all()
            self.assign(centers)
            new_centers = self.calculate_new_centers()
            print (new_centers)
        return
            

kmeans = KMeans(X, 6)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)

[[ 0.5561198   0.76904028]
 [ 2.37269287 -2.75214461]
 [-2.75338814  1.58980866]
 [ 0.51110491 -4.05502333]
 [ 2.40258879  2.16073643]
 [ 2.14063769 -4.93881809]]
[[ 0.29025049  0.53361458]
 [ 2.36980244 -2.71084867]
 [-2.96683383  1.73918315]
 [ 0.48165758 -3.89288144]
 [ 2.19929155  2.18102419]
 [ 2.12725699 -4.92918475]]
[[ 0.0823844   0.32130635]
 [ 2.40726929 -2.70907402]
 [-3.04580724  1.80226156]
 [ 0.54051231 -3.99136321]
 [ 2.05414874  2.12402626]
 [ 2.16266388 -4.89711892]]
[[-0.04533652  0.20009658]
 [ 2.38454477 -2.67063319]
 [-3.07703402  1.85266958]
 [ 0.54051231 -3.99136321]
 [ 1.97627167  2.05817317]
 [ 2.23312425 -4.83830617]]
[[-0.09853317  0.11682125]
 [ 2.38454477 -2.67063319]
 [-3.07703402  1.85266958]
 [ 0.6085265  -4.08710448]
 [ 1.93298349  2.03064159]
 [ 2.23312425 -4.83830617]]
[[-0.13586639  0.09262377]
 [ 2.38454477 -2.67063319]
 [-3.07703402  1.85266958]
 [ 0.67992746 -4.18869934]
 [ 1.91248297  2.00204407]
 [ 2.33846326 -4.83215045]]
[[-0.13586639  0.09262